# How to use HSDPython_SDK HSD_utils module - \[Data format conversion Features\]
---
<br>
<p>Import HSDatalog and HSDatalogConverter modules</p>

In [ ]:
from HSD_utils.converters import HSDatalogConverter
from HSD.HSDatalog import HSDatalog

HSDatalog will be only used to extract Pandas dataframes in a simple way (extracting them from an HSDatalog acquisition folder), as shown in the [nb_hsdatalog_core.ipynb] notebook.

In [ ]:
acq_folder = "..\\STWIN_acquisition_examples\\STWIN_00002"
hsd = HSDatalog(acquisition_folder=acq_folder)
sensor_name = "IIS3DWB"
sensor_type = "ACC"
dataframe = hsd.get_dataframe(sensor_name, sensor_type)
print(dataframe)

# Data Conversion
---
<p>The HSDatalogConverter module works independently from the core module, as long as the data to be converted are Pandas DataFrames.</p>
Starting from a DataFrame it is possible to convert the data of an acquisition in the following formats:
<ul>
<li> CSV, TSV
<li> Cartesiam format
<li> ST UNICO format
<li> WAV (Only for Microphones)
</ul>
To access the conversion features, we have to:
<ul>
<li> Import the HSD_utils.converters module (already done in the first code cell).
<li> choose a folder to save the conversion output files
</ul>


In [ ]:
from HSD_utils.converters import HSDatalogConverter
import os

output_folder = "..\\STWIN_acquisition_examples\\STWIN_00002\\exported_data_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

## CSV, TSV Conversion

In [ ]:
sensor_name = "HTS221"
sensor_type = "TEMP"
dataframe = hsd.get_dataframe(sensor_name, sensor_type)

exported_sensor_path = os.path.join(output_folder,sensor_name)

print("CSV Conversion Started")
HSDatalogConverter.to_csv(dataframe, exported_sensor_path)
print("CSV Conversion Ended")
print("TSV Conversion Started")
HSDatalogConverter.to_tsv(dataframe, exported_sensor_path)
print("TSV Conversion Ended")

## Cartesiam format Conversion

In [ ]:
sensor_name = "IIS3DWB"
sensor_type = "ACC"
dataframe = hsd.get_dataframe(sensor_name, sensor_type)

HSDatalogConverter.to_cartesiam_format(output_folder, sensor_name, dataframe, signal_length = 1000, signal_increment = 750)

## ST UNICO format Conversion
Available outpuf formats currently accepted in input by Unico GUI: txt (default), csv, tsv. 
- Unlabeled acquisition. Even if present, the tag data in the acquisition is not used

In [ ]:
hsd_dfs = []
sensor_name = "ISM330DHCX"
sensor_type = "GYRO"
hsd_dfs.append(hsd.get_dataframe(sensor_name, sensor_type))

HSDatalogConverter.to_unico(output_folder, sensor_name, hsd_dfs, out_format = "csv")

- Labeled Acquisition. If available, the tag data are read and used to split the converted files

In [ ]:
hsd_dfs = []
sensor_name = "ISM330DHCX"
sensor_type = "GYRO"
hsd_dfs.append(hsd.get_dataframe(sensor_name, sensor_type, labeled = True))

data_tags = hsd.get_data_stream_tags(sensor_name, sensor_type)

HSDatalogConverter.to_unico(output_folder, sensor_name, hsd_dfs, data_tags, "csv")

## WAV Conversion
- single shot conversion

In [ ]:
import numpy as np
from IPython.display import Audio

acq_folder = "..\\STWIN_acquisition_examples\\STWIN_20210205_15_47_16"
hsd = HSDatalog(acquisition_folder=acq_folder)
output_folder = "..\\STWIN_acquisition_examples\\STWIN_20210205_15_47_16\\exported_data_folder"
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

sensor_name = "IMP23ABSU"
sensor = hsd.get_sensor(sensor_name)
odr = sensor.sensor_status.sub_sensor_status[0].odr
print("ODR: {}".format(odr))
wav_file_name = os.path.join(output_folder, "{}_MIC.wav".format(sensor.name))
audio_data = hsd.get_data_and_timestamps(sensor_name,'MIC')[0]
pcm_data = audio_data.astype(np.int16)
HSDatalogConverter.to_wav(pcm_data,wav_file_name,odr)
Audio(wav_file_name)

- Batch processing (useful for large files)

In [ ]:
import numpy as np
from IPython.display import Audio

sensor_name = "IMP34DT05"
sensor = hsd.get_sensor(sensor_name)
odr = sensor.sensor_status.sub_sensor_status[0].odr
print("ODR: {}".format(odr))
wav_file_name = os.path.join(output_folder, "{}_MIC.wav".format(sensor.name))
wav_file = HSDatalogConverter.wav_create(wav_file_name, odr)

chunkSize = 10000000
isLastChunk = False
sample_offset = 0
while isLastChunk == 0:
    data = hsd.get_data_and_timestamps(sensor.name, "MIC", sample_start = sample_offset, sample_end = sample_offset+chunkSize, raw_flag = True)[0]  
    pcm_data = data.astype(np.int16)
    if len(data) < chunkSize:
        isLastChunk = 1
    else:
        sample_offset = sample_offset + chunkSize
    HSDatalogConverter.wav_append(wav_file,pcm_data)
HSDatalogConverter.wav_close(wav_file)
Audio(wav_file_name)